In [5]:
# Import Flask
from flask import Flask, jsonify

In [6]:
# Dependencies and Setup
import numpy as np
import datetime as dt

In [7]:
# Python SQL Toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [8]:
#Setting up the engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [9]:
# reflect an existing database into a new model
Base = automap_base()

In [10]:
# Reflect the Tables
Base.prepare(engine, reflect=True)

In [11]:
# Save reference to the table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [12]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)

In [22]:
@app.route("/")
def welcome():
        return """<html>
    <h1>Hawaii Climate App (Flask API)</h1>
    <img src="https://i.ytimg.com/vi/3ZiMvhIO-d4/maxresdefault.jpg" alt="Hawaii Weather"/>
    <p>Precipitation Analysis:</p>
        <ul>
    <li><a href="/api/v1.0/precipitation">/api/v1.0/precipitation</a></li>
        </ul>
    <p>Station Analysis:</p>
        <ul>
      <li><a href="/api/v1.0/stations">/api/v1.0/stations</a></li>
        </ul>
    <p>Temperature Analysis:</p>
        <ul>
      <li><a href="/api/v1.0/tobs">/api/v1.0/tobs</a></li>
    </ul>
    <p>Start Day Analysis:</p>
        <ul>
      <li><a href="/api/v1.0/2017-03-14">/api/v1.0/2017-03-14</a></li>
        </ul>
    <p>Start & End Day Analysis:</p>
        <ul>
      <li><a href="/api/v1.0/2017-03-14/2017-03-28">/api/v1.0/2017-03-14/2017-03-28</a></li>
        </ul>
    </html>
    """

In [14]:
# Precipitation Route
@app.route("/api/v1.0/precipitation")
def precipitation():
        # Convert the Query Results to a Dictionary Using `date` as the Key and `prcp` as the Value
        # Calculate the Date 1 Year Ago from the Last Data Point in the Database
        one_year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
        # Design a Query to Retrieve the Last 12 Months of Precipitation Data Selecting Only the `date` and `prcp` Values
        prcp_data = session.query(Measurement.date, Measurement.prcp).\
                filter(Measurement.date >= one_year_ago).\
                order_by(Measurement.date).all()
        # Convert List of Tuples Into a Dictionary
        prcp_data_list = dict(prcp_data)
        # Return JSON Representation of Dictionary
        return jsonify(prcp_data_list)

In [15]:
# Station Route
@app.route("/api/v1.0/stations")
def stations():
        # Return a JSON List of Stations From the Dataset
        stations_all = session.query(Station.station, Station.name).all()
        # Convert List of Tuples Into Normal List
        station_list = list(stations_all)
        # Return JSON List of Stations from the Dataset
        return jsonify(station_list)


In [16]:
# TOBs Route
@app.route("/api/v1.0/tobs")
def tobs():
        # Query for the Dates and Temperature Observations from a Year from the Last Data Point
        one_year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
        # Design a Query to Retrieve the Last 12 Months of Precipitation Data Selecting Only the `date` and `prcp` Values
        tobs_data = session.query(Measurement.date, Measurement.tobs).\
                filter(Measurement.date >= one_year_ago).\
                order_by(Measurement.date).all()
        # Convert List of Tuples Into Normal List
        tobs_data_list = list(tobs_data)
        # Return JSON List of Temperature Observations (tobs) for the Previous Year
        return jsonify(tobs_data_list)


In [17]:
# Start Day Route
@app.route("/api/v1.0/<start>")
def start_day(start):
        start_day = session.query(Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
                filter(Measurement.date >= start).\
                group_by(Measurement.date).all()
        # Convert List of Tuples Into Normal List
        start_day_list = list(start_day)
        # Return JSON List of Min Temp, Avg Temp and Max Temp for a Given Start Range
        return jsonify(start_day_list)


In [ ]:
if __name__ == "__main__":
    app.run(debug=True)